# Kaggle Dogs vs. Cats Redux: Kernels Edition
## My Convolutional Neural Networks Implementation
Task: Distinguish images of dogs from cats 

matplotlib is a python 2D plotting library which produces publication quality figures in a variety of hardcopy formats and interactive environments across platforms. (www.matplotlib.org)

In [2]:
%matplotlib inline

Path to data:

In [3]:
#path = "data/dogs-cats-redux-data/"
path = "data/dogs-cats-redux-data/sample/"

Libraries

In [5]:
from __future__ import division,print_function

import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt

import utils; reload(utils)
from utils import plots

Using Theano backend.


# Use of Pretrained VGG Model

Use a model that has been fully created and which can recognise a wide variety (1,000 categories) of images. 'VGG' won the 2014 Imagenet competition and is a simple model to create and understand. The Python class *Vgg16* offers the functioality to use the VGG 16 model in a straightforward manner.

In [7]:
# Import vgg16 class and instantiate
import vgg16; reload(vgg16)
from vgg16 import Vgg16

vgg = Vgg16()

MemoryError: 